In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel("internship_simplified.xlsx")
df

,Time,T° Read 1:472,A1,A2,A3,A4,A5,A6,A7,A8,...,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12
0,00:00:07,25.1,0.0696,0.0789,0.0764,0.0753,0.0709,0.0732,0.0753,0.0723,...,0.0694,0.0749,0.0786,0.0793,0.0719,0.0757,0.0664,0.0764,0.0718,0.0724
1,00:01:07,25.0,0.0711,0.0757,0.0745,0.0743,0.0739,0.0755,0.0717,0.0731,...,0.0702,0.0783,0.0821,0.0825,0.0725,0.0758,0.0684,0.0780,0.0727,0.0724
2,00:02:07,25.0,0.0734,0.0781,0.0773,0.0762,0.0753,0.0789,0.0712,0.0725,...,0.0704,0.0804,0.0860,0.0862,0.0722,0.0760,0.0722,0.0783,0.0733,0.0727
3,00:03:07,25.0,0.0753,0.0803,0.0796,0.0786,0.0777,0.0814,0.0715,0.0715,...,0.0705,0.0840,0.0901,0.0903,0.0727,0.0766,0.0722,0.0790,0.0735,0.0732
4,00:04:07,25.0,0.0776,0.0823,0.0817,0.0810,0.0808,0.0836,0.0726,0.0718,...,0.0709,0.0871,0.0941,0.0941,0.0730,0.0769,0.0730,0.0794,0.0739,0.0738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,01:28:07,25.2,0.2248,0.2324,0.2439,0.2360,0.2380,0.2566,0.1180,0.0853,...,0.0874,0.2630,0.3121,0.3006,0.0850,0.1178,0.1067,0.1152,0.0992,0.1228
89,01:29:07,25.2,0.2265,0.2340,0.2451,0.2374,0.2394,0.2581,0.1186,0.0856,...,0.0876,0.2643,0.3140,0.3021,0.0851,0.1162,0.1072,0.1156,0.0996,0.1234
90,01:30:07,25.2,0.2278,0.2351,0.2465,0.2387,0.2408,0.2594,0.1187,0.0857,...,0.0877,0.2658,0.3156,0.3036,0.0853,0.1169,0.1076,0.1160,0.0998,0.1239
91,01:31:07,25.2,0.2286,0.2368,0.2480,0.2402,0.2421,0.2609,0.1195,0.0859,...,0.0878,0.2671,0.3175,0.3055,0.0854,0.1207,0.1079,0.1165,0.1003,0.1245


In [13]:
def check_times(current_well, special_wells, special_tmin, special_tmax, df):
    if current_well in special_wells:
        ind = special_wells.index(current_well)
        _ , _ , filt_sig = ts.get_start_time(df['time'].values, df[current_well].values)
        loc_tmin = special_tmin[ind]
        loc_tmax = special_tmax[ind]
    else:
        loc_tmin, loc_tmax, filt_sig = ts.get_start_time(df['time'].values, df[current_well].values)
    
    return loc_tmin, loc_tmax, pd.Series(filt_sig)

In [14]:
%matplotlib qt

In [15]:
nplot_rows = 8   # these parameters are standard for a 96 well-plate
nplot_cols = 12  # and only of interest for plotting purposes
well_names = df.columns[2:] # collect the well coordinates in this array

# FIRST figure: absorption vs time
figs, axs = plt.subplots(nplot_rows,nplot_cols, figsize=(24, 16), layout='constrained')
for ax, c in zip(axs.flat,well_names):
    ax.plot(df.Time,df[c])
    ax.text(0.05,0.95,c,transform=ax.transAxes, fontsize=16, verticalalignment='top')
            
    loc_tmin, loc_tmax, filt_sig = check_times(c,special_wells,special_tmin,special_tmax,df)
    #ax.plot(df.time,filt_sig)
    ax.plot(df.time,filt_sig)
            
    rate_calcs = reaction_rate([df.time,filt_sig],loc_tmin,loc_tmax)
    rate = rate_calcs[0]
    t = np.array(df.time).astype(np.float64)
    bool_list = (t>=loc_tmin)
    i0 = bool_list.argmax()
    t0 = t[i0]
    y0 = df[c][i0]
            
    ax.plot(t,rate*(t-t0)+y0)
            
    ax.text(0.5,0.05,"{:.2f}".format(float(rate/norm_rate)),transform=ax.transAxes, fontsize=16, verticalalignment='bottom',color='red')
plt.show()

TypeError: float() argument must be a string or a real number, not 'datetime.time'

In [ ]:
x = df['Time'].apply(lambda t: t.hour * 3600 + t.minute * 60 + t.second).to_numpy()
# Convert Time to proper time format (optional but good for plotting)
#df['Time'] = pd.to_timedelta(df['Time'])

# Get all columns except 'Time'
y_columns = df.columns[2:]
n = len(y_columns)

# Create subplots
#fig, axes = plt.subplots(8,12, figsize=(16,25), sharex=True)
#axes = axes.flatten()

#for i, col in enumerate(y_columns):
#    axes[i].plot(x, y_columns, marker='o')

#plt.tight_layout()
#plt.show()


ValueError: x and y must have same first dimension, but have shapes (93,) and (96,)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import string

# Assume df has columns: Time, A1..A12, B1..B12, ..., H1..H12
# And Time is already processed into seconds or suitable numeric form

rows, cols = 8, 12
letters = list(string.ascii_uppercase[:rows])  # ['A', 'B', ..., 'H']

fig, axes = plt.subplots(rows, cols, figsize=(20, 15), sharex=True, sharey=True)
axes = axes.reshape((rows, cols))

for i, letter in enumerate(letters):
    for j in range(1, cols + 1):
        ax = axes[i, j - 1]
        col_name = f"{letter}{j}"
        
        if col_name in df.columns:
            ax.plot(x, df[col_name], color='blue')
            ax.set_title(col_name, fontsize=8)
        else:
            ax.set_visible(False)

        ax.tick_params(labelsize=6)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Create time axis
time = np.arange(0, 93 * 7, 7)  # 93 time points, 7-second intervals

# Generate column names: A1–A12, B1–B12, ..., H1–H12
columns = [f"{row}{col}" for row in "ABCDEFGH" for col in range(1, 13)]

# Simulate data: linear trend + noise
data = {
    col: (
        np.random.uniform(0.05, 0.1) +            # baseline
        np.random.uniform(1e-5, 5e-5) * time +     # slope
        np.random.normal(0, 0.001, size=len(time)) # noise
    )
    for col in columns
}

# Build DataFrame
df = pd.DataFrame(data)
df.insert(0, "Time", time)



# df is now ready to use


Index(['Time', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10',
       'A11', 'A12', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9',
       'B10', 'B11', 'B12', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8',
       'C9', 'C10', 'C11', 'C12', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7',
       'D8', 'D9', 'D10', 'D11', 'D12', 'E1', 'E2', 'E3', 'E4', 'E5', 'E6',
       'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'F1', 'F2', 'F3', 'F4', 'F5',
       'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'G1', 'G2', 'G3', 'G4',
       'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11', 'G12', 'H1', 'H2', 'H3',
       'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10', 'H11', 'H12'],
      dtype='object')

In [17]:
df

,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12
0,0,0.078423,0.068338,0.085478,0.075077,0.053473,0.092869,0.081571,0.069922,0.100055,...,0.064510,0.057898,0.086287,0.082115,0.070981,0.080923,0.064314,0.082125,0.071681,0.062408
1,7,0.077652,0.068297,0.084838,0.072921,0.053366,0.092096,0.083390,0.071797,0.098918,...,0.065825,0.060208,0.086769,0.084657,0.068586,0.081138,0.064082,0.083107,0.071207,0.063267
2,14,0.078995,0.069059,0.086598,0.073880,0.055471,0.092826,0.081481,0.071794,0.097843,...,0.064749,0.059306,0.084740,0.082714,0.071245,0.081397,0.064752,0.083196,0.071984,0.063176
3,21,0.079111,0.068556,0.087314,0.072933,0.053263,0.093163,0.082059,0.073688,0.096534,...,0.065294,0.060371,0.087183,0.084274,0.069821,0.081462,0.062703,0.083281,0.070801,0.061187
4,28,0.079315,0.068856,0.086932,0.073414,0.054908,0.094280,0.081299,0.073258,0.097626,...,0.065922,0.059974,0.086964,0.084749,0.070917,0.082147,0.064152,0.083113,0.071535,0.063740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,616,0.089513,0.092244,0.111653,0.091298,0.067700,0.102552,0.104226,0.098753,0.120851,...,0.072448,0.082998,0.112593,0.110255,0.094326,0.108212,0.069461,0.099892,0.087661,0.089344
89,623,0.091215,0.094461,0.112090,0.091948,0.067778,0.100976,0.104450,0.100935,0.120741,...,0.073063,0.082535,0.113339,0.107809,0.093399,0.107866,0.070077,0.101840,0.085704,0.087681
90,630,0.091174,0.094245,0.112500,0.090226,0.067506,0.101959,0.104753,0.100924,0.123764,...,0.071668,0.082469,0.111885,0.108043,0.094616,0.108351,0.071556,0.103391,0.087700,0.087559
91,637,0.092084,0.094407,0.113425,0.092308,0.066439,0.101147,0.104497,0.103203,0.122509,...,0.073050,0.084211,0.114870,0.109688,0.095120,0.110230,0.071217,0.102965,0.086545,0.089369
